<a href="https://colab.research.google.com/github/KJTDHISAW/Fine_Tuning_PlinAI/blob/main/Fine_Tuning_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print('Mahna Mahna')

Mahna Mahna


##Imports

In [2]:
import nltk
import pandas as pd
import re
from tabulate import tabulate
import numpy as np
import spacy
from spacy import displacy
from spacy.lang.la import Latin

## Load Drive

In [3]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
!ls /content/drive/MyDrive

'Colab Notebooks'


## Load CLTK

In [ ]:
# I have had some ongoing trouble with getting la_core_web_lg to work alongside the cltk_readers
# There seems to be a spacy verson incompatability
#(Thus far skipping this step works, but sometimes if cltk isn't
# working on my comp for some reason I'll just download la_core_web_lg and a
# bunch of processing can be achieved anyway.)
#!pip install https://huggingface.co/latincy/la_core_web_lg/resolve/main/la_core_web_lg-any-py3-none-any.whl

In [ ]:
# You might have to restart runtime - I do everytime I start a session fresh
#!pip install spacy==3.6.0
#!pip install -e git+https://github.com/diyclassics/cltk_readers.git#egg=cltk_readers

In [ ]:
#!pip install roman #there are still some modern critical edition roman numerals
#in Pliny, but also a number of numbers given by Pliny, so I'm working to see
#how best to preprocess the text

In [4]:
import la_core_web_lg
nlp = la_core_web_lg.load()

In [5]:
from cltkreaders.lat import LatinTesseraeCorpusReader

CR = LatinTesseraeCorpusReader()
CR_fileids = CR.fileids(match='pliny_the_elder')
CR_fileids

CLTK message: Unless a path is specifically passed to the 'root' parameter, this corpus reader expects to find the CLTK-Tesserae texts at lat/text/lat_text_tesserae/texts.
Do you want to download CLTK-Tesserae Latin files? [Y/n] 
Y


INFO:CLTK:Cloning 'lat_text_tesserae' from 'https://github.com/cltk/lat_text_tesserae.git'


INFO:CLTK:Cloning 'lat_models_cltk' from 'https://github.com/cltk/lat_models_cltk.git'


['pliny_the_elder.naturalis_historia.part.1.books_1-5.tess',
 'pliny_the_elder.naturalis_historia.part.2.books_6-10.tess',
 'pliny_the_elder.naturalis_historia.part.3.books_11-15.tess',
 'pliny_the_elder.naturalis_historia.part.4.books_16-20.tess',
 'pliny_the_elder.naturalis_historia.part.5.books_21-25.tess',
 'pliny_the_elder.naturalis_historia.part.6.books_26-30.tess',
 'pliny_the_elder.naturalis_historia.part.7.books_31-37.tess']

## Preprocess text

In [6]:
data = []

def process_tess_line(line):
    line = re.sub(r'\s', ' ', line)

    match = re.match(r'<([^<>]+)>\s*([^<>]+(?:\s*\d*\s*[^<>]+)*)', line)

    if match:
        label = match.group(1)
        sentence = match.group(2)

        latin_chars_only = re.sub(r'[^\x00-\x7F]+', ' ', sentence)
        clean_sentence = re.sub(r'[^\w\s]', '', latin_chars_only)

        doc = nlp(clean_sentence)
        lemmas_list = [token.lemma_ for token in doc]
        pos_list = [token.pos_ for token in doc]

        line_data = {'label': label, 'sentence': clean_sentence, 'lemmas': lemmas_list, 'pos': pos_list}
        data.append(line_data)

for fileid in CR_fileids:
    file_content = CR.raw(fileids=[fileid])
    lines = file_content.split('\n')

    for line in lines:
        process_tess_line(line)

plin_df7 = pd.DataFrame(data)


In [7]:
print(plin_df7)

                 label                                           sentence  \
0       plin. nat. 1.1  libro ii continentvr an finitus sit mundus et ...   
1       plin. nat. 1.2  ex avctoribvs M Varrone sulpicio gallo tito ca...   
2       plin. nat. 1.3  externis hipparcho timaeo sosigene petosiri ne...   
3       plin. nat. 1.4  l iii continentvr situs gentes maria oppida po...   
4       plin. nat. 1.5  ex avctoribvs turranio gracile cornelio nepote...   
...                ...                                                ...   
2852  plin. nat. 37.74  veras a falsis discernere magna difficultas qu...   
2853  plin. nat. 37.75  gemmiferi amnes sunt acesinus et ganges terrar...   
2854  plin. nat. 37.76  ergo in toto orbe quacumque caeli convexitas v...   
2855  plin. nat. 37.77  rerum autem ipsarum maximum est pretium in mar...   
2856  plin. nat. 37.78  salve parens rerum omnium natura teque nobis q...   

                                                 lemmas  \
0     [liber, is

In [8]:
empty_rows = plin_df7[plin_df7.isna().any(axis=1)]
print(empty_rows[['label', 'sentence', 'lemmas', 'pos']])


Empty DataFrame
Columns: [label, sentence, lemmas, pos]
Index: []


In [9]:
plin_df7.to_csv('/content/drive/MyDrive/plin_df7.csv', index=False, encoding='utf-8')

## CV and TFIDF dataset

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

plin_df7['joined_lemmas'] = plin_df7['lemmas'].apply(lambda lemmas: ' '.join(lemmas))

def filter_tokens(doc, unwanted_pos=['ADP', 'CCONJ', 'PUNCT', "PRON", "AUX", "DET"]):
    return [token.lemma_ for token in doc if token.pos_ not in unwanted_pos]

plin_df7['filtered_joined_lemmas'] = plin_df7['joined_lemmas'].apply(lambda lemmas: ' '.join(filter_tokens(nlp(lemmas))))

custom_token_pattern = r'\b[^\W\d_]+\b'
cv = CountVectorizer(token_pattern=custom_token_pattern)
cv_matrix = cv.fit_transform(plin_df7['filtered_joined_lemmas'])
cv_feature_names = cv.get_feature_names_out()
cv_df = pd.DataFrame(cv_matrix.toarray(), columns=cv_feature_names)

print(cv_df)

      a  aacroceraunius  aadvesperaveos  aaeuus  aaevum  aaliterua  \
0     0               0               0       0       0          0   
1     0               0               0       0       0          0   
2     0               0               0       0       0          0   
3     0               0               0       0       0          0   
4     0               0               0       0       0          0   
...  ..             ...             ...     ...     ...        ...   
2852  0               0               0       0       0          0   
2853  0               0               0       0       0          0   
2854  0               0               0       0       0          0   
2855  0               0               0       0       0          0   
2856  0               0               0       0       0          0   

      aaparctias  aapis  aapodas  aaraxos  ...  zoster  zotha  zoton  zulilus  \
0              0      0        0        0  ...       0      0      0        0 

In [11]:
cv_df.to_csv('/content/drive/MyDrive/plin_cv_df7.csv', index=False, encoding='utf-8')

In [12]:
print(plin_df7[['label', 'sentence', 'lemmas', 'pos', 'joined_lemmas', 'filtered_joined_lemmas']])

                 label                                           sentence  \
0       plin. nat. 1.1  libro ii continentvr an finitus sit mundus et ...   
1       plin. nat. 1.2  ex avctoribvs M Varrone sulpicio gallo tito ca...   
2       plin. nat. 1.3  externis hipparcho timaeo sosigene petosiri ne...   
3       plin. nat. 1.4  l iii continentvr situs gentes maria oppida po...   
4       plin. nat. 1.5  ex avctoribvs turranio gracile cornelio nepote...   
...                ...                                                ...   
2852  plin. nat. 37.74  veras a falsis discernere magna difficultas qu...   
2853  plin. nat. 37.75  gemmiferi amnes sunt acesinus et ganges terrar...   
2854  plin. nat. 37.76  ergo in toto orbe quacumque caeli convexitas v...   
2855  plin. nat. 37.77  rerum autem ipsarum maximum est pretium in mar...   
2856  plin. nat. 37.78  salve parens rerum omnium natura teque nobis q...   

                                                 lemmas  \
0     [liber, is

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

custom_token_pattern = r'\b[^\W\d_]+\b'
tfidf_vectorizer = TfidfVectorizer(token_pattern=custom_token_pattern)
tfidf_matrix = tfidf_vectorizer.fit_transform(plin_df7['filtered_joined_lemmas'])
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_feature_names)

print(tfidf_df)

        a  aacroceraunius  aadvesperaveos  aaeuus  aaevum  aaliterua  \
0     0.0             0.0             0.0     0.0     0.0        0.0   
1     0.0             0.0             0.0     0.0     0.0        0.0   
2     0.0             0.0             0.0     0.0     0.0        0.0   
3     0.0             0.0             0.0     0.0     0.0        0.0   
4     0.0             0.0             0.0     0.0     0.0        0.0   
...   ...             ...             ...     ...     ...        ...   
2852  0.0             0.0             0.0     0.0     0.0        0.0   
2853  0.0             0.0             0.0     0.0     0.0        0.0   
2854  0.0             0.0             0.0     0.0     0.0        0.0   
2855  0.0             0.0             0.0     0.0     0.0        0.0   
2856  0.0             0.0             0.0     0.0     0.0        0.0   

      aaparctias  aapis  aapodas  aaraxos  ...  zoster  zotha  zoton  zulilus  \
0            0.0    0.0      0.0      0.0  ...     0.0

In [ ]:
tfidf_df.to_csv('/content/drive/MyDrive/plin_tfidf_df7.csv', index=False, encoding='utf-8')


## CV and TFIDF TOP 10,5,3 TERMS

In [14]:
def get_top_terms(row, n_top_terms):
    return row.sort_values(ascending=False).index[:n_top_terms].tolist()

n_top_terms = 10

cv_top_terms_column = [get_top_terms(row, n_top_terms) for _, row in cv_df.iterrows()]
cv_top_terms_column = np.array(cv_top_terms_column)
plin_df7['cv_top_terms'] = cv_top_terms_column.tolist()

tfidf_top_terms_column = [get_top_terms(row, n_top_terms) for _, row in tfidf_df.iterrows()]
tfidf_top_terms_column = np.array(tfidf_top_terms_column)
plin_df7['tfidf_top_terms'] = tfidf_top_terms_column.tolist()

In [15]:
print(plin_df7[['label', 'sentence', 'lemmas', 'pos', 'joined_lemmas', 'filtered_joined_lemmas', 'cv_top_terms', 'tfidf_top_terms']])

                 label                                           sentence  \
0       plin. nat. 1.1  libro ii continentvr an finitus sit mundus et ...   
1       plin. nat. 1.2  ex avctoribvs M Varrone sulpicio gallo tito ca...   
2       plin. nat. 1.3  externis hipparcho timaeo sosigene petosiri ne...   
3       plin. nat. 1.4  l iii continentvr situs gentes maria oppida po...   
4       plin. nat. 1.5  ex avctoribvs turranio gracile cornelio nepote...   
...                ...                                                ...   
2852  plin. nat. 37.74  veras a falsis discernere magna difficultas qu...   
2853  plin. nat. 37.75  gemmiferi amnes sunt acesinus et ganges terrar...   
2854  plin. nat. 37.76  ergo in toto orbe quacumque caeli convexitas v...   
2855  plin. nat. 37.77  rerum autem ipsarum maximum est pretium in mar...   
2856  plin. nat. 37.78  salve parens rerum omnium natura teque nobis q...   

                                                 lemmas  \
0     [liber, is

In [16]:
plin_df7.to_csv('/content/drive/MyDrive/plin_top10_df7.csv', index=False, encoding='utf-8')

In [17]:
plin_df7['author'] = 'pliny_the_elder'

In [18]:
empty_rows = plin_df7[plin_df7.isna().any(axis=1)]
print(empty_rows[['label', 'sentence', 'lemmas', 'pos']])

Empty DataFrame
Columns: [label, sentence, lemmas, pos]
Index: []


##Formatting dataset into q/a


In [ ]:
!pip install -q 'gradio>=4.0.0' # you can probably remove this if you're running locally (KT: I find that I have to run this each time)
import gradio as gr
from huggingface_hub import InferenceClient

In [ ]:
model = "mistralai/Mixtral-8x7B-Instruct-v0.1" # see https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1

client = InferenceClient(model) # see https://huggingface.co/docs/huggingface_hub/package_reference/inference_client

# If you haven't set HF_TOKEN as a secret for your notebooks, you'll get a warning. It can be ignored.

In [27]:
from transformers import AutoTokenizer

model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)


/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

### First KT prompt try

In [ ]:
data_for_training = []

for index, row in plin_df7.iterrows():
    label = row['label']
    sentence = row['sentence']
    author = row['author']

    for method, top_terms in [('CountVectorizer', row['cv_top_terms']), ('TF-IDF', row['tfidf_top_terms'])]:
        for term in top_terms:
            question = f"What does {author} in {label} say about {term} according to {method}?"
            answer = sentence

            conversation_input = [
                tokenizer.bos_token,
                tokenizer.encode("[INST]") + tokenizer.encode(question) + tokenizer.encode("[/INST]"),
                tokenizer.encode(answer) + [tokenizer.eos_token_id],
            ]

            data_point = {
                'instruction': question,
                'expected_output': answer,
                'conversation_input': conversation_input,
            }

            data_for_training.append(data_point)


### Second SH prompt suggestion try

In [40]:
data_for_training = []

for index, row in plin_df7.iterrows():
    label = row['label']
    sentence = row['sentence']
    author = row['author']

    for method, top_terms in [('CountVectorizer', row['cv_top_terms']), ('TF-IDF', row['tfidf_top_terms'])]:
        for term in top_terms:
            # Create the question and expected result for each top_term
            question = f"Identify and then include in your response a passage of Pliny that addresses the following topic: {term} according to {method}"
            expected_result = f"At {label}, {author} wrote the following: {sentence}"

            conversation_input = [
                tokenizer.bos_token,
                tokenizer.encode("[INST]") + tokenizer.encode(question) + tokenizer.encode("[/INST]"),
                tokenizer.encode(expected_result) + [tokenizer.eos_token_id],
            ]

            data_point = {
                'instruction': question,
                'expected_output': expected_result,
                'conversation_input': conversation_input,
            }

            data_for_training.append(data_point)


## Save/load training data

In [41]:
# Save data_for_training as needed
df_for_training = pd.DataFrame(data_for_training)
df_for_training.to_csv('/content/drive/MyDrive/training_data7_SH_prompt.csv', index=False, encoding='utf-8')

In [42]:
import torch
from torch.utils.data import Dataset
import pandas as pd
import pyarrow as pa

class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.data = self._process_dataframe()

    def _process_dataframe(self):
        data_for_training = []

        for index, row in self.dataframe.iterrows():
            text = '<s>[INST]@Enlighten. ' + row['instruction'] + '[/INST]' + row['expected_output'] + '</s>'
            conversation_input = [
                self.tokenizer.bos_token_id,
                *self.tokenizer.encode("[INST]@Enlighten. " + row['instruction'] + "[/INST]" + row['expected_output']),
                self.tokenizer.eos_token_id,
            ]

            data_point = {
                'text': text,
                'conversation_input': conversation_input,
            }

            data_for_training.append(data_point)

        return data_for_training

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]


# Create the CustomDataset using the manually prepared data_for_training
custom_dataset = CustomDataset(dataframe=pd.DataFrame(data_for_training), tokenizer=tokenizer)

In [43]:
# Print the first few items in custom_dataset
for i in range(min(5, len(custom_dataset))):
    print(custom_dataset[i])


{'text': '<s>[INST]@Enlighten. Identify and then include in your response a passage of Pliny that addresses the following topic: terra according to CountVectorizer[/INST]At plin. nat. 1.1, pliny_the_elder wrote the following: libro ii continentvr an finitus sit mundus et an unus de forma eius de motu eius cur mundus dicatur de elementis de deo de siderum errantium natura de lunae et solis defectibus de nocte de magnitudine siderum quae quis invenerit in observatione caelesti de lunae motu errantium motus et luminum canonica quare eadem altiora alias propiora videantur catholica siderum errantium quae ratio colores eorum mutet solis motus dierum inaequalitatis ratio quare fulmina iovi adsignentur intervalla siderum de sideribus musica de mundo geometrica de repentinis sideribus de cometis natura et situs et genera eorum hipparchea de sideribus agnoscendis  de caelestibus prodigiis per exempla historica lampades bolides trabes caelestes chasma caeli de caeli coloribus de flamma caelesti 

In [45]:
len(custom_dataset)

57140